In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
#installation de ngrok
# si pas de compte sur ngrok : creer un compte et remplacer l'authotoken par le token à recuperer sur le compte creer
!pip install flask-ngrok
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok   
!ngrok authtoken "23jQLlx658T8GxNtoOtdgGzz9Nc_5zBATxLx64qAL8zzcg7Li"

deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia

In [21]:
#Fonction qui prepare l'environnement Yolo
def prepare_environment():
  !git clone https://github.com/AlexeyAB/darknet
  %cd darknet
  !sed -i 's/OPENCV=0/OPENCV=1/' Makefile
  !sed -i 's/GPU=0/GPU=1/' Makefile
  !sed -i 's/CUDNN=0/CUDNN=1/' Makefile
  !sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
  !/usr/local/cuda/bin/nvcc --version
  !make

  #copier les zip obj et test a la racine des fichiers colab
  !cp /content/drive/Shareddrives/Projet\ Deep\ Learning/'Deep Learning'/obj.zip ../
  !cp /content/drive/Shareddrives/Projet\ Deep\ Learning/'Deep Learning'/test.zip ../

  # /darknet/data/ 
  !unzip ../obj.zip -d data/
  !unzip ../test.zip -d data/

  # upload the custom .cfg back to cloud VM from Google Drive
  !cp /content/drive/Shareddrives/Projet\ Deep\ Learning/'Deep Learning'/yolov4-obj.cfg ./cfg

  # upload the obj.names and obj.data files to cloud VM from Google Drive
  !cp /content/drive/Shareddrives/Projet\ Deep\ Learning/'Deep Learning'/obj.names ./data
  !cp /content/drive/Shareddrives/Projet\ Deep\ Learning/'Deep Learning'/obj.data  ./data

  # need to set our custom cfg to test mode 
  !sed -i 's/batch=64/batch=1/' /content/drive/Shareddrives/Projet\ Deep\ Learning/'Deep Learning'/yolov4-obj.cfg
  !sed -i 's/subdivisions=16/subdivisions=1/' /content/drive/Shareddrives/Projet\ Deep\ Learning/'Deep Learning'/yolov4-obj.cfg


In [22]:
def detect(filen):
  !./darknet detector test /content/drive/Shareddrives/Projet\ Deep\ Learning/'Deep Learning'/obj.data /content/drive/Shareddrives/Projet\ Deep\ Learning/'Deep Learning'/yolov4-obj.cfg /content/drive/Shareddrives/Projet\ Deep\ Learning/'Deep Learning'/yolov4-obj_last.weights -dont_show -ext_output /content/drive/Shareddrives/Projet\ Deep\ Learning/'Deep Learning'/{filen} > /content/drive/Shareddrives/Projet\ Deep\ Learning/'Deep Learning'/output.txt 
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  import re
  with open("/content/drive/Shareddrives/Projet Deep Learning/Deep Learning/output.txt", "r") as file:
    first_line = file.readline()
    for last_line in file:
      pass
  last_line = last_line.replace("license_plate:","").replace("left_x:","").replace("top_y:","").replace("width:","").replace("height:","")
  last_line = last_line.split("(")
  last_line1 = last_line[1].split("     ")
  last_line1[0] = last_line1[0].replace("  ","")
  last_line1[3] = last_line1[3].replace("\n","").replace(")","").replace(" ","")
  last_line1[2] = last_line1[2].replace(" ","")
  left_x = int(last_line1[0])
  top_y = int(last_line1[1])
  width = int(last_line1[2])
  height = int(last_line1[3])
  img = cv2.imread('predictions.jpg')
  roi = img[top_y:top_y+height,left_x:left_x+width]
  return roi

In [23]:
prepare_environment()

fatal: destination path 'darknet' already exists and is not an empty directory.
[Errno 20] Not a directory: 'darknet'
/content/darknet
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
           

In [24]:
cd /content/drive/Shareddrives/Projet Deep Learning/Deep Learning/Lecture de caractères/rendu

/content/drive/Shareddrives/Projet Deep Learning/Deep Learning/Lecture de caractères/rendu


In [25]:
from flask import Flask, request, render_template, jsonify
from flask_ngrok import run_with_ngrok
import os
import urllib.request
from werkzeug.utils import secure_filename
import segment_plate_characters
import pickle
from cv2 import imread
from cv2 import imshow
import matplotlib.pyplot as plt
from PIL import Image 

app = Flask(__name__)
 
UPLOAD_FOLDER = 'static/uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
 
app.secret_key = "cairocoders-tutorial101"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
 
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'PNG'])
VIDEO_EXTENSION = set(['mp4', 'MP4'])

 
def allowed_file_image(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def allowed_file_video(filename):
  return '.' in filename and filename.rsplit('.', 1)[1].lower() in VIDEO_EXTENSION
 
@app.route('/')
def upload_form():
    return render_template('index.html')
     
@app.route('/upload', methods=['POST'])
def upload_file():
    # check if the post request has the file part
    if 'files[]' not in request.files:
        resp = jsonify({'message' : 'No file part in the request'})
        resp.status_code = 400
        return resp
     
    files = request.files.getlist('files[]')
     
    errors = {}
    success = False
     
    for file in files:
        # cas image
        if file and allowed_file_image(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            success = True

        #cas video
        if file and allowed_file_video(file.filename):
           filename = secure_filename(file.filename)
           file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
           img_vid = segment_plate_characters.capturer_image(os.path.join(app.config['UPLOAD_FOLDER'], filename) )
           filename = 'image.jpg'
           plt.imsave(os.path.join(app.config['UPLOAD_FOLDER'], filename), img_vid)
           success = True

        if (success==False):
            errors[file.filename] = 'File type is not allowed'
     
    if success and errors:
        errors['message'] = 'File(s) successfully uploaded'
        resp = jsonify(errors)
        resp.status_code = 206
        return resp
    if success:
        path = os.getcwd()
        character_recognizing_model_path = path + '/modele/character_recognizing_model.sav'
        model = pickle.load(open(character_recognizing_model_path, 'rb'))

        image_voiture  =  imread(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        !ls static/uploads/
        !cp -r static/uploads/{filename} /content/drive/Shareddrives/Projet\ Deep\ Learning/Deep\ Learning/
        %cd /content/darknet
        imge  = detect(filename)
        plt.imsave('/content/drive/Shareddrives/Projet Deep Learning/Deep Learning/p1.jpg',imge)
        plate_image = imread("/content/drive/Shareddrives/Projet Deep Learning/Deep Learning/p1.jpg")

        %cd /content/drive/Shareddrives/Projet Deep Learning/Deep Learning/Lecture de caractères/rendu
        result = segment_plate_characters.predict_license_number(model, plate_image)
        print('res',result)
        resp = jsonify({'message' : 'Le numéro de la plaque est  :  <span style="color:red">'+result+'<span>'})
        resp.status_code = 201
        return resp
    else:
        resp = jsonify(errors)
        resp.status_code = 400
        return resp
 
run_with_ngrok(app)  
app.run()